In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import sys
import random
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Math
from torch.utils.data import DataLoader, Dataset
import time
import json
from scipy.integrate import trapz

from utils import *
from model_utils import *
#from model_VAE import VAE

sys_epsilon = sys.float_info.epsilon

--------------------

# Data Preparation

In [2]:
headers = ["t",                                             # time
           "X", "Y", "Z",                                   # spacial coordinates
           "Ux", "Uy", "Uz",                                # velocity components
           "G1", "G2", "G3", "G4", "G5", "G6",              # velocity gradient tensor components
           "S1", "S2", "S3", "S4", "S5", "S6",              # strain rate tensor compnents
           "UUp1", "UUp2", "UUp3", "UUp4", "UUp5", "UUp6",  # resolved Reynolds stress tensor components
           "Cs"]                                            # Smagorinsky coefficient

### Dataset Loading

#### $\mathcal Re = 10^3$

In [3]:
# Re = 10^3 seen
with open('../processedDatasets/fieldData_R3_seen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R103_seen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R3_seen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R103_seen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen_norm.txt', sep=' ', names=headers)
dS_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen.txt', sep=' ', names=headers)

# Re = 10^3 unseen
with open('../processedDatasets/fieldData_R3_unseen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R103_unseen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R3_unseen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R103_unseen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen_norm.txt', sep=' ', names=headers)
dS_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 5\times 10^3$

In [4]:
# Re = 5 x 10^3 seen
with open('../processedDatasets/fieldData_R53_seen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R503_seen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R53_seen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R503_seen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen_norm.txt', sep=' ', names=headers)
dS_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen.txt', sep=' ', names=headers)

# Re = 5 x 10^3 unseen
with open('../processedDatasets/fieldData_R53_unseen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R503_unseen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R53_unseen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R503_unseen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen_norm.txt', sep=' ', names=headers)
dS_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 10^4$

In [5]:
# Re = 10^4 seen
with open('../processedDatasets/fieldData_R4_seen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R104_seen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R4_seen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R104_seen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen_norm.txt', sep=' ', names=headers)
dS_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen.txt', sep=' ', names=headers)

# Re = 10^4 unseen
with open('../processedDatasets/fieldData_R4_unseen_means.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R104_unseen_means = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

with open('../processedDatasets/fieldData_R4_unseen_scales.txt', 'r') as file:
    data = [float(line.strip()) for line in file]
dSn_R104_unseen_scales = pd.DataFrame(np.reshape(data, (-1, len(headers))), columns=headers)

dSn_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen_norm.txt', sep=' ', names=headers)
dS_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen.txt', sep=' ', names=headers)

--------------------

# Model Configuration Setup

In this section, we consider the different model configurations as presented in the following table.

$$
\begin{array}{|l|c|c|c|c|}
    \hline
    \textbf{Model} & \textbf{Inputs} & \textbf{No. of Inputs} & \textbf{Outputs} & \textbf{No. of Outputs} \\
    \hline
    \mathbf{M1} & u_i \, \text{and} \, \mathcal{S}_{ij} & 9 & c_s & 1\\
    \mathbf{M2} & \mathcal{G}_{ij} \, \text{and} \, \mathcal{S}_{ij} & 12 & c_s & 1 \\
    \mathbf{M3} & u_i \, \text{and} \, \tau^{'}_{ij} & 9 & c_s & 1 \\
    \mathbf{M4} & \mathcal{G}_{ij} \, \text{and} \, \tau^{'}_{ij} & 12 & c_s & 1 \\
    \hline
\end{array}
$$

In [13]:
M1_headers = ['Ux', 'Uy', 'Uz', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M2_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M3_headers = ['Ux', 'Uy', 'Uz', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']
M4_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']

M1_103 = dSn_R103_seen.filter(M1_headers, axis=1)
M2_103 = dSn_R103_seen.filter(M2_headers, axis=1)
M3_103 = dSn_R103_seen.filter(M3_headers, axis=1)
M4_103 = dSn_R103_seen.filter(M4_headers, axis=1)

M1_503 = dSn_R503_seen.filter(M1_headers, axis=1)
M2_503 = dSn_R503_seen.filter(M2_headers, axis=1)
M3_503 = dSn_R503_seen.filter(M3_headers, axis=1)
M4_503 = dSn_R503_seen.filter(M4_headers, axis=1)

M1_104 = dSn_R104_seen.filter(M1_headers, axis=1)
M2_104 = dSn_R104_seen.filter(M2_headers, axis=1)
M3_104 = dSn_R104_seen.filter(M3_headers, axis=1)
M4_104 = dSn_R104_seen.filter(M4_headers, axis=1)

M1_103_test = dSn_R103_unseen.filter(M1_headers, axis=1)
M2_103_test = dSn_R103_unseen.filter(M2_headers, axis=1)
M3_103_test = dSn_R103_unseen.filter(M3_headers, axis=1)
M4_103_test = dSn_R103_unseen.filter(M4_headers, axis=1)

M1_503_test = dSn_R503_unseen.filter(M1_headers, axis=1)
M2_503_test = dSn_R503_unseen.filter(M2_headers, axis=1)
M3_503_test = dSn_R503_unseen.filter(M3_headers, axis=1)
M4_503_test = dSn_R503_unseen.filter(M4_headers, axis=1)

M1_104_test = dSn_R104_unseen.filter(M1_headers, axis=1)
M2_104_test = dSn_R104_unseen.filter(M2_headers, axis=1)
M3_104_test = dSn_R104_unseen.filter(M3_headers, axis=1)
M4_104_test = dSn_R104_unseen.filter(M4_headers, axis=1)

--------------------

# Model Training

In [14]:
dt = M4_104
dt_name = namestr(M4_104, globals())[0]

In [15]:
split_sz = 0.8
mask = np.random.rand(len(dt)) < split_sz
train = dt[mask].reset_index(drop=True) 
val = dt[~mask].reset_index(drop=True)

In [16]:
train

,G1,G2,G3,G4,G5,G6,UUp1,UUp2,UUp3,UUp4,UUp5,UUp6,Cs
0,-0.030288,-0.099401,-0.093050,-0.044349,0.073241,0.071077,-0.621343,0.005343,0.001508,-0.727846,0.024628,-0.723760,-1.004127
1,-0.197695,-0.242680,0.244637,0.835817,-1.329357,0.323531,-0.576310,-0.179442,0.034037,-0.476491,-0.439450,-0.646736,0.517645
2,-0.030994,0.652822,-0.045647,-0.148220,0.266144,-0.168707,-0.249698,-0.002098,-0.543235,-0.440977,-0.039008,-0.175606,-1.158358
3,-0.181946,-0.020944,-0.226529,-0.082291,0.444896,0.366172,-0.426727,0.319302,0.126813,-0.350229,0.865285,-0.541787,-0.686264
4,-0.671802,-0.761704,0.023589,0.006468,-0.524679,-0.105963,-0.353570,-0.196120,0.414301,-0.421726,-1.029824,-0.323891,0.131864
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560013,0.497272,-0.566571,-3.453885,-0.884785,-0.153910,1.150560,2.288330,-2.898927,4.222684,4.088178,0.486467,3.742809,0.653598
560014,0.210802,0.010281,1.138833,0.914302,0.851010,-1.320538,-0.137856,0.121102,-0.623054,-0.325073,-0.726222,-0.028926,0.372382
560015,0.719513,-0.389316,-0.419565,-0.152395,-0.146498,-0.100599,-0.333556,0.240537,-0.080920,-0.096904,-0.444976,-0.156379,-0.127999
560016,0.183035,0.044767,0.739180,0.015846,-0.085614,-0.129985,-0.465270,0.158376,-0.265106,-0.476840,-0.927122,-0.356582,-0.469827


In [17]:
val

,G1,G2,G3,G4,G5,G6,UUp1,UUp2,UUp3,UUp4,UUp5,UUp6,Cs
0,-0.229502,-0.082355,-0.049943,0.005463,0.058222,-0.131098,-0.206289,0.149457,0.584462,-0.410026,0.766261,-0.168835,-0.289756
1,-0.618960,-0.582423,-0.004830,0.100815,-0.213630,-0.717968,-0.046226,-0.724693,0.063926,-0.055390,-0.104848,-0.493566,0.266733
2,0.750543,0.366205,0.141528,-0.943148,-0.561064,0.235249,-0.141035,0.570926,0.142963,0.271379,0.462132,0.058082,-0.044033
3,0.663552,0.894064,0.500579,0.237269,0.854063,0.226380,-0.026175,-0.170825,-0.723355,0.051490,0.990494,0.350830,-0.274570
4,-0.098686,-0.093137,0.030419,0.007388,0.050987,0.196056,-0.535607,-0.161492,-0.081420,-0.493975,0.608716,-0.589099,-0.503224
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139977,-1.148371,-0.213732,1.626341,0.943446,0.194335,0.498926,-0.135747,-0.040028,-0.633390,-0.332281,0.261262,-0.022080,-0.182028
139978,0.874507,0.107767,0.525375,-0.254840,-0.467373,-2.643559,1.235610,-0.862587,3.120241,2.032296,-1.978951,2.847636,0.345989
139979,0.606251,-1.046614,-0.152563,-0.098721,-0.953079,-0.064235,-0.014469,0.883996,0.483994,0.099337,1.341227,-0.175988,0.220853
139980,-0.016059,-0.503635,-0.137585,-0.190025,-0.146906,0.418550,-0.133851,0.294660,0.613317,-0.278234,1.259872,-0.099662,0.093227


In [18]:
batch_sz_trn = 4096
batch_sz_val = int(batch_sz_trn / 4)

train_dataset = MyDataset(train)
val_dataset = MyDataset(val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_sz_trn, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_sz_val, shuffle=True)

In [19]:
data_iter = iter(train_loader)
next(data_iter)[0]

tensor([ 0.1266, -0.0371, -0.2418,  0.0321, -0.1173, -0.5373, -0.2734, -0.3704,
         0.3557, -0.2544, -1.2157, -0.2559, -0.2192], dtype=torch.float64)

In [26]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, output_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim * 2)  # Output mu and logvar
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)  # Output a single value C_s
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        # Encode
        h = self.encoder(x)
        mu, logvar = h[:, :latent_dim], h[:, latent_dim:]
        z = self.reparameterize(mu, logvar)
        
        # Decode
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, logvar

output_size = 1
input_size = dt.shape[1] - output_size 
latent_size = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

vae = VAE(input_size, latent_size, output_size)
vae.to(device)
vae.double()
optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)


In [37]:
def mse_loss_function(recon_x, x, mu, logvar):
    # Calculate reconstruction loss (MSE)
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')
    
    # Calculate KL divergence
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + kl_divergence

In [38]:
num_epochs = 20
vae.train()

for epoch in range(num_epochs):
    train_loss = 0
    for batch in train_loader:
        inputs = batch[:, :-output_size].to(device)  # Use only the input features
        targets = batch[:, -output_size:].to(device)  # Use the output value C_s
        optimizer.zero_grad()
        recon_targets, mu, logvar = vae(inputs)
        loss = mse_loss_function(recon_targets, targets, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}, Loss: {train_loss / len(train_loader)}")

# Save the trained model
torch.save(vae.state_dict(), 'vae_wavelet.pth')

# Evaluate the VAE
vae.eval()
val_loss = 0
with torch.no_grad():
    for batch in val_loader:
        inputs = batch[:, :-output_size].to(device)  # Use only the input features
        targets = batch[:, -output_size:].to(device)  # Use the output value C_s
        recon_targets, mu, logvar = vae(inputs)
        loss = mse_loss_function(recon_targets, targets, mu, logvar)
        val_loss += loss.item()

print(f"Validation Loss: {val_loss / len(val_loader)}")


Epoch 1, Loss: 4068.6894736265226
Epoch 2, Loss: 4066.9142523023356
Epoch 3, Loss: 4066.684731762904


KeyboardInterrupt: 

----------------------

# Model Analysis

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

## $\mathcal Re = 10^3$

In [ ]:
datasets = {
    'M1_103': M1_103,
    'M2_103': M2_103,
    'M3_103': M3_103,
    'M4_103': M4_103
}

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Loop through each dataset and perform calculations and plotting
for i, (dt_name, dt) in enumerate(datasets.items()):
    print(f"--- Using this Model Config: {dt_name}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"--- Running on {device}!")

    PATH = f"./best_model_{dt_name}.pt"
    output_size = 1
    input_size = dt.shape[1] - output_size
    neurons_per_layer = [60, 60, 60, 60, 60]
    hidden_layers = len(neurons_per_layer)
    model = MLPModel(input_size=input_size,
                     output_size=output_size,
                     hidden_layers=hidden_layers,
                     neurons_per_layer=neurons_per_layer)

    model.load_state_dict(torch.load(PATH))
    model.eval()
    model.to(device)
    model.double()

    test = globals()[f"{dt_name}_test"]
    test_features = test.iloc[:, :-1].values
    test_label = test.iloc[:, -1].values
    test_features_tensor = torch.tensor(test_features).double().to(device)

    Cs_norm_pred = model(test_features_tensor)
    Cs_norm = Cs_norm_pred.detach().cpu().numpy()
    Cs_tilde = Cs_norm * dSn_R103_unseen_scales['Cs'].values + dSn_R103_unseen_means['Cs'].values
    Cs_GT = test_label * dSn_R103_unseen_scales['Cs'].values + dSn_R103_unseen_means['Cs'].values

    n, xedges, yedges = np.histogram2d(Cs_GT, Cs_tilde.squeeze(), bins=[1500, 1501])
    jpdf = n / trapz(trapz(n, xedges[:-1], axis=0), yedges[:-1])
    X, Y = np.meshgrid(xedges[:-1], yedges[:-1])

    ax = axes[i // 2, i % 2]
    c = ax.pcolormesh(X, Y, jpdf.T, shading='auto', cmap='jet')
    c.set_clim([-4, 54])
    ax.set_title(f'$\mathbf M{dt_name[1]}$', fontsize=16)
    ax.set_xlabel(r'$C_s$', fontsize=14)
    ax.set_ylabel(r'$\tilde{C_s}$', fontsize=14)
    ax.set_xlim([-0.15, 0.15])
    ax.set_ylim([-0.15, 0.15])
    fig.colorbar(c, ax=ax)

plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Loop through each dataset and perform calculations and plotting
for i, (dt_name, dt) in enumerate(datasets.items()):
    print(f"--- Using this Model Config: {dt_name}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"--- Running on {device}!")

    PATH = f"./best_model_{dt_name}.pt"
    output_size = 1
    input_size = dt.shape[1] - output_size
    neurons_per_layer = [60, 60, 60, 60, 60]
    hidden_layers = len(neurons_per_layer)
    model = MLPModel(input_size=input_size,
                     output_size=output_size,
                     hidden_layers=hidden_layers,
                     neurons_per_layer=neurons_per_layer)

    model.load_state_dict(torch.load(PATH))
    model.eval()
    model.to(device)
    model.double()

    test = globals()[f"{dt_name}_test"]
    test_features = test.iloc[:, :-1].values
    test_label = test.iloc[:, -1].values
    test_features_tensor = torch.tensor(test_features).double().to(device)

    Cs_norm_pred = model(test_features_tensor)
    Cs_norm = Cs_norm_pred.detach().cpu().numpy()
    Cs_tilde = Cs_norm * dSn_R103_unseen_scales['Cs'].values + dSn_R103_unseen_means['Cs'].values
    Cs_GT = test_label * dSn_R103_unseen_scales['Cs'].values + dSn_R103_unseen_means['Cs'].values

    ax = axes[i // 2, i % 2]
    ax.scatter(Cs_GT, Cs_tilde.squeeze(), edgecolor='white', color='red')
    ax.set_title(f'{dt_name}', fontsize=14)
    ax.set_xlabel(r'$C_s$', fontsize=12)
    ax.set_ylabel(r'$\tilde{C_s}$', fontsize=12)
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])

plt.tight_layout()
plt.show()


--------------------

## $\mathcal Re = 5 \times 10^3$

In [ ]:
datasets = {
    'M1_503': M1_503,
    'M2_503': M2_503,
    'M3_503': M3_503,
    'M4_503': M4_503
}

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Loop through each dataset and perform calculations and plotting
for i, (dt_name, dt) in enumerate(datasets.items()):
    print(f"--- Using this Model Config: {dt_name}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"--- Running on {device}!")

    PATH = f"./best_model_{dt_name}.pt"
    output_size = 1
    input_size = dt.shape[1] - output_size
    neurons_per_layer = [60, 60, 60, 60, 60]
    hidden_layers = len(neurons_per_layer)
    model = MLPModel(input_size=input_size,
                     output_size=output_size,
                     hidden_layers=hidden_layers,
                     neurons_per_layer=neurons_per_layer)

    model.load_state_dict(torch.load(PATH))
    model.eval()
    model.to(device)
    model.double()

    test = globals()[f"{dt_name}_test"]
    test_features = test.iloc[:, :-1].values
    test_label = test.iloc[:, -1].values
    test_features_tensor = torch.tensor(test_features).double().to(device)

    Cs_norm_pred = model(test_features_tensor)
    Cs_norm = Cs_norm_pred.detach().cpu().numpy()
    Cs_tilde = Cs_norm * dSn_R503_unseen_scales['Cs'].values + dSn_R503_unseen_means['Cs'].values
    Cs_GT = test_label * dSn_R503_unseen_scales['Cs'].values + dSn_R503_unseen_means['Cs'].values

    n, xedges, yedges = np.histogram2d(Cs_GT, Cs_tilde.squeeze(), bins=[1500, 1501])
    jpdf = n / trapz(trapz(n, xedges[:-1], axis=0), yedges[:-1])
    X, Y = np.meshgrid(xedges[:-1], yedges[:-1])

    ax = axes[i // 2, i % 2]
    c = ax.pcolormesh(X, Y, jpdf.T, shading='auto', cmap='jet')
    c.set_clim([-4, 54])
    ax.set_title(f'$\mathbf M{dt_name[1]}$', fontsize=16)
    ax.set_xlabel(r'$C_s$', fontsize=14)
    ax.set_ylabel(r'$\tilde{C_s}$', fontsize=14)
    ax.set_xlim([-0.15, 0.15])
    ax.set_ylim([-0.15, 0.15])
    fig.colorbar(c, ax=ax)

plt.tight_layout()
plt.show()


--------------------

## $\mathcal Re = 10^4$

In [ ]:
datasets = {
    'M1_104': M1_104,
    'M2_104': M2_104,
    'M3_104': M3_104,
    'M4_104': M4_104
}

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Loop through each dataset and perform calculations and plotting
for i, (dt_name, dt) in enumerate(datasets.items()):
    print(f"--- Using this Model Config: {dt_name}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"--- Running on {device}!")

    PATH = f"./best_model_{dt_name}.pt"
    output_size = 1
    input_size = dt.shape[1] - output_size
    neurons_per_layer = [60, 60, 60, 60, 60]
    hidden_layers = len(neurons_per_layer)
    model = MLPModel(input_size=input_size,
                     output_size=output_size,
                     hidden_layers=hidden_layers,
                     neurons_per_layer=neurons_per_layer)

    model.load_state_dict(torch.load(PATH))
    model.eval()
    model.to(device)
    model.double()

    test = globals()[f"{dt_name}_test"]
    test_features = test.iloc[:, :-1].values
    test_label = test.iloc[:, -1].values
    test_features_tensor = torch.tensor(test_features).double().to(device)

    Cs_norm_pred = model(test_features_tensor)
    Cs_norm = Cs_norm_pred.detach().cpu().numpy()
    Cs_tilde = Cs_norm * dSn_R104_unseen_scales['Cs'].values + dSn_R104_unseen_means['Cs'].values
    Cs_GT = test_label * dSn_R104_unseen_scales['Cs'].values + dSn_R104_unseen_means['Cs'].values

    n, xedges, yedges = np.histogram2d(Cs_GT, Cs_tilde.squeeze(), bins=[1500, 1501])
    jpdf = n / trapz(trapz(n, xedges[:-1], axis=0), yedges[:-1])
    X, Y = np.meshgrid(xedges[:-1], yedges[:-1])

    ax = axes[i // 2, i % 2]
    c = ax.pcolormesh(X, Y, jpdf.T, shading='auto', cmap='jet')
    c.set_clim([-4, 54])
    ax.set_title(f'$\mathbf M{dt_name[1]}$', fontsize=16)
    ax.set_xlabel(r'$C_s$', fontsize=14)
    ax.set_ylabel(r'$\tilde{C_s}$', fontsize=14)
    ax.set_xlim([-0.15, 0.15])
    ax.set_ylim([-0.15, 0.15])
    fig.colorbar(c, ax=ax)

plt.tight_layout()
plt.show()
